In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
df_comment = pd.DataFrame(columns=["Comment UID",
                                   "Post UID",
                                   "Topic",
                                   "Link",
                                   "Comment Floor Number",
                                   "Number of Like",
                                   "Comment Create Date",
                                   "Comment Create Time",
                                   "Comment Author",
                                   "Comment Author Score",
                                   "Comment Text"
                                  ])

df_post = pd.read_excel("BabyKingdom_Bank_Data.xlsx", index_col=None)
p_topic = df_post["Topic"].values.tolist()
p_uid = df_post["Post_UID"].values.tolist()
links = df_post["Link"].values.tolist()

test_links = []
test_links.append(links[0])
test_links.append(links[1])
test_links.append(links[273])
test_links.append(links[272])
test_links.append(links[271])
test_links.append(links[270])

In [3]:
print(links[272])

https://www.baby-kingdom.com/forum.php?mod=viewthread&tid=1220568


In [4]:
#setup engine for web scraping
options = webdriver.ChromeOptions()
#path of the chrome driver 
chromedriver_path='chromedriver.exe'
#open the chrome driver
driver = webdriver.Chrome(options=options, executable_path=chromedriver_path) 

C:\Users\User\AppData\Local\Temp/ipykernel_21212/2448344236.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=chromedriver_path)


In [5]:
def scrape_comment_1st_pg(soup,topic,link,post_uid):
    floor_check = 0
    try:
        for i in soup.find_all("div",{"class":"postthread"}):
            floor = i.find_all("em")[3].text
            time = i.find_all("span")[1].text
            date = time.split(' ')[0]
            created_time = time.split(' ')[1]

            author = i.find_all("a")[1].text
            if author == "":
                author = i.find_all("a")[2].text

            score = i.find_all("p")[1].text.split(" ")[1]

            temp_text = []
            for j in i.find_all("span"):
                if j.text != '|' and j.text != '' and j.text != '\n':
                    temp_text.append(j.text)
            temp_text.pop(0)
            temp_text = ''.join(temp_text)

            if floor_check == 0 :
                likes = i.find("span",{"id":"recommendv_add"}).text
                floor_check = 1
            else:
                likes = 0

            #create comment UID
            # BK-00000001-00001
            # platform-post_number-comment_number
            comment_uid = "-"+str(floor).zfill(5)
            new_comment_uid = post_uid+comment_uid

            df_comment.loc[len(df_comment)]= [new_comment_uid, post_uid, topic,link, floor, likes, date, created_time, author, score, temp_text]
    except:
        pass

def scrape_comment_2nd_pg(soup,topic,link,post_uid):
    floor_check = 0
    try:
        for i in soup.find_all("div",{"class":"postthread"}):
            floor = i.find_all("em")[3].text
            time = i.find_all("span")[1].text
            date = time.split(' ')[0]
            created_time = time.split(' ')[1]

            author = i.find_all("a")[1].text
            if author == "":
                author = i.find_all("a")[2].text

            score = i.find_all("p")[1].text.split(" ")[1]

            temp_text = []
            for j in i.find_all("span"):
                if j.text != '|' and j.text != '' and j.text != '\n':
                    temp_text.append(j.text)
            temp_text.pop(0)
            temp_text = ''.join(temp_text)
            likes = 0

            #create comment UID
            # BK-00000001-00001
            # platform-post_number-comment_number
            comment_uid = "-"+str(floor).zfill(5)
            new_comment_uid = post_uid+comment_uid

            df_comment.loc[len(df_comment)]= [new_comment_uid, post_uid, topic,link, floor, likes, date, created_time, author, score, temp_text]
    except:
        pass


In [6]:
for idx, link in enumerate(links):
    try:
        post_uid = p_uid[idx]
        topic = p_topic[idx]
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html)

        #scrape first page
        #pass org title uid and author
        scrape_comment_1st_pg(soup,topic,link,post_uid)

        #if "next page" is available

        while soup.find("i",{"class":"ico_on icon-pagin-generic_nextPageOn"}) != None:
            if soup.find('i',{"class":"ico_off icon-pagin-generic_nextPageOn"}) != None:
                break 
            nextpage = driver.find_element(By.CLASS_NAME,"icon-pagin-generic_nextPageOn")
            nextpage.click()
            window_after = driver.window_handles[0]
            driver.switch_to.window(window_after)
            html = driver.page_source
            soup =BeautifulSoup(html)
            scrape_comment_2nd_pg(soup,topic,link,post_uid)

        print(f"completed {idx} links")
    except:
        pass
print("End...")

completed 0 links
completed 1 links
completed 2 links
completed 3 links
completed 4 links
completed 5 links
completed 6 links
completed 7 links
completed 8 links
completed 9 links
completed 10 links
completed 11 links
completed 12 links
completed 13 links
completed 14 links
completed 15 links
completed 16 links
completed 17 links
completed 18 links
completed 19 links
completed 20 links
completed 21 links
completed 22 links
completed 23 links
completed 24 links
completed 25 links
completed 26 links
completed 27 links
completed 28 links
completed 29 links
completed 30 links
completed 31 links
completed 32 links
completed 33 links
completed 34 links
completed 35 links
completed 36 links
completed 37 links
completed 38 links
completed 39 links
completed 40 links
completed 41 links
completed 42 links
completed 43 links
completed 44 links
completed 45 links
completed 46 links
completed 47 links
completed 48 links
completed 49 links
completed 50 links
completed 51 links
completed 52 links
com

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.063
  (Session info: chrome=108.0.5359.99)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BEACD3+2075859]
	Ordinal0 [0x00B7EE61+1633889]
	Ordinal0 [0x00A7B7BD+571325]
	Ordinal0 [0x00A6E410+517136]
	Ordinal0 [0x00A6E231+516657]
	Ordinal0 [0x00A6D0A3+512163]
	Ordinal0 [0x00A6D6D8+513752]
	Ordinal0 [0x00A76662+550498]
	Ordinal0 [0x00A81BD5+596949]
	Ordinal0 [0x00A84FC0+610240]
	Ordinal0 [0x00A6DA81+514689]
	Ordinal0 [0x00A81A0F+596495]
	Ordinal0 [0x00ADB93B+964923]
	Ordinal0 [0x00AC7136+880950]
	Ordinal0 [0x00A9FEFD+720637]
	Ordinal0 [0x00AA0F3F+724799]
	GetHandleVerifier [0x00E9EED2+2769538]
	GetHandleVerifier [0x00E90D95+2711877]
	GetHandleVerifier [0x00C7A03A+521194]
	GetHandleVerifier [0x00C78DA0+516432]
	Ordinal0 [0x00B8682C+1665068]
	Ordinal0 [0x00B8B128+1683752]
	Ordinal0 [0x00B8B215+1683989]
	Ordinal0 [0x00B96484+1729668]
	BaseThreadInitThunk [0x7526FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x771B7BBE+286

In [7]:
#Close the driver...
driver.quit() 
print(f"End of scraping {len(df_comment)} has been scraped...")

End of scraping 184840 has been scraped...


In [8]:
df_comment

,Comment UID,Post UID,Topic,Link,Comment Floor Number,Number of Like,Comment Create Date,Comment Create Time,Comment Author,Comment Author Score,Comment Text
0,BK-00000000-00001,BK-00000000,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封,https://www.baby-kingdom.com/forum.php?mod=vie...,1,0,20-1-16,10:45,Flora3k,3220,本帖最後由 Flora3k 於 21-1-12 07:24 編輯 \n\n中國銀行/滙豐/...
1,BK-00000000-00002,BK-00000000,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封,https://www.baby-kingdom.com/forum.php?mod=vie...,2,0,20-1-18,13:29,Flora3k,3220,Flora3k 發表於 20-1-16 10:45 \n中國銀行/滙豐/恒生/周大福/渣打利...
2,BK-00000000-00003,BK-00000000,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封,https://www.baby-kingdom.com/forum.php?mod=vie...,3,0,20-1-18,13:30,Flora3k,3220,Flora3k 發表於 20-1-18 13:29 \n補相\n補相\n\n\n\n20-1...
3,BK-00000000-00004,BK-00000000,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封,https://www.baby-kingdom.com/forum.php?mod=vie...,4,0,20-1-19,12:00,Flora3k,3220,Flora3k 發表於 20-1-18 13:30 \n補相\n大華銀行/恒生銀行/渣打銀行...
4,BK-00000000-00005,BK-00000000,中國銀行/滙豐/恒生/周大福/Citi Bank/渣打利是封,https://www.baby-kingdom.com/forum.php?mod=vie...,5,0,21-1-12,07:24,Flora3k,3220,Up\n
...,...,...,...,...,...,...,...,...,...,...,...
184835,BK-00012207-00047,BK-00012207,HSBC Premier,https://www.baby-kingdom.com/forum.php?mod=vie...,47,0,12-7-30,07:36,maggie625,15258,原帖由 彥彥媽咪 於 12-07-27 發表\n本帖最後由 彥彥媽咪 於 12-7-27 0...
184836,BK-00012207-00048,BK-00012207,HSBC Premier,https://www.baby-kingdom.com/forum.php?mod=vie...,48,0,12-7-30,14:21,cherryho1021,10278,咁都好似唔係咁so 喎...
184837,BK-00012207-00049,BK-00012207,HSBC Premier,https://www.baby-kingdom.com/forum.php?mod=vie...,49,0,12-7-30,16:45,ccyv,11392,我覺得無咩用，個經理只係識叫你買基金，優惠又無乜特別，最有用係個外幣對換率高啲，有時匯款唔收...
184838,BK-00012207-00050,BK-00012207,HSBC Premier,https://www.baby-kingdom.com/forum.php?mod=vie...,50,0,12-7-30,16:45,ccyv,11392,我覺得無咩用，個經理只係識叫你買基金，優惠又無乜特別，最有用係個外幣對換率高啲，有時匯款唔收...


In [9]:
#output data
df_comment.to_excel('BabyKingdom_Bank_Post_Comment_Data.xlsx',encoding= 'utf-8-sig',index = None) 

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
C:\Users\User\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://www.baby-kingdom.com/forum.php?mod=viewthread&tid=22596117' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\User\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://www.baby-kingdom.com/forum.php?mod=viewthread&tid=22557149' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
C:\Users\User\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1177: UserWarning: Ignoring URL 'https://www.baby-kingdom.com/forum.php?mod=viewthread&tid=22563349' since i

In [10]:
#output data
df_comment.to_excel('BabyKingdom_Bank_Post_Comment_Data_backup.xlsx',index = None) 